In [103]:
from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/My\ Drive/UROP/project/UROP_project/

ModuleNotFoundError: No module named 'google.colab'

In [104]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [105]:
files = utils.files_in_order('poses_compressed/squat')

X_train_names, X_test_names = train_test_split(files[5:16], test_size=0.2, random_state=42)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)

pprint(X_train_names)
pprint(y_train)
pprint(X_test_names)
pprint(y_test)

array(['squat_good_7.npy', 'squat_bad_8.npy', 'squat_bad_7.npy',
       'squat_good_5.npy', 'squat_good_1.npy', 'squat_good_4.npy',
       'squat_bad_9.npy', 'squat_good_3.npy'], dtype='<U17')
array([1, 0, 0, 1, 1, 1, 0, 1])
array(['squat_good_2.npy', 'squat_bad_6.npy', 'squat_good_6.npy'],
      dtype='<U17')
array([1, 0, 1])


In [116]:
# scaling 후에 정해진 위치로 shift
def normalization(pose) :
    # lenght normalization
    # torso 길이가 1이 되도록 scale param을 구하고 resize
    fixed_torso_legnth = 1 # tosro 길이를 1로 설정
    
    torso_lengths = np.array([dist(pose.neck, pose.lhip)] +
                                 [dist(pose.neck, pose.rhip)])
    mean_torso = np.mean(torso_lengths)
    scale = fixed_torso_legnth / mean_torso
    
    #print("\n\nScale")
    #print(scale)
    
    # shift
    # 2, 2로 맞추기
    set_x, set_y = 2, 2
    x, y = pose.neck.x*scale, pose.neck.y*scale
    
    shift_x = ((set_x-x)*(set_x-x))**0.5
    shift_y = ((set_y-y)*(set_y-y))**0.5
    
    if set_x < x : 
        shift_x = -shift_x
    if set_y < y : 
        shift_y = -shift_y
    
    #print("SHIFT")
    #print(shift_x,shift_y)
    #print(x + shift_x, y + shift_y)

    return scale, shift_x, shift_y

def dist(part1, part2):
    return np.sqrt(np.square(part1.x - part2.x) + np.square(part1.y - part2.y))


In [117]:
import matplotlib.pyplot as plt

def plot_keypoints(x, y) :
    plt.xlim([0,5])
    plt.ylim([0,5])
    plt.scatter(x, y)
    plt.show()

    plt.rcdefaults() 
    

In [118]:
def cal_vector_change(x_array, y_array) : 
    result = np.array([])
    
    for t in range(1, len(x_array)) :
        tmp = []
        v = np.square(x_array[t-1] - x_array[t])
        w = np.square(y_array[t-1] - y_array[t])

        result = np.append(result, np.sum(np.sqrt(v+w)))
 
    #print(result.shape)
    return result

In [119]:
def calculate_change_keypoints(names, flag, mode='right'):
    # 촬영 방향
    # 오른쪽에서 촬영(오른쪽 관절만 관여)
    if mode == 'right' :
        PART_NAMES = ['neck', 'rshoulder', 'relbow', 'rwrist', 'rhip', 'rknee', 'rankle']
    

    # 왼쪽에서 촬영(왼쪽 관절만 관여)
    elif mode == 'left' :
        PART_NAMES = ['neck', 'lshoulder', 'lelbow', 'lwrist', 'lhip', 'lknee', 'lankle']

    # 앞 또는 뒤에서 촬영(모든 관절이 관여)
    else :
        PART_NAMES = ['neck',  'rshoulder', 'relbow', 'rwrist', 'rhip', 'rknee', 'rankle',
                      'lshoulder', 'lelbow', 'lwrist', 'lhip', 'lknee', 'lankle']
    
    result = []
    for filename in names:
        output = []
        x_array = []
        y_array = []

        #tmp_np = np.load('poses_compressed/squat/'+filename)

        if flag == 0: # flag 0 이면 기존 training, test 
            ps = load_ps('poses_compressed/squat/'+filename)

        # (!! 수정필요 - 새로운 파일 돌릴 수 있도록)
        else: # 1 flag 이면 새로 만든 값 
            ps = load_ps('poses_compressed/squat/'+filename)
        
        poses = ps.poses
        
        for pose in poses : 
            x = []
            y = []
            
            scale, shift_x, shift_y = normalization(pose)

            for part in PART_NAMES :
                # if getattr(pose, part).x < 0 : continue
                #print(part)
                #print(getattr(pose, part).x * scale + shift_x, getattr(pose, part).y * scale + shift_y)
                x.append(getattr(pose, part).x * scale + shift_x)
                y.append(getattr(pose, part).y * scale + shift_y)
                  
            x_array.append(x)
            y_array.append(y)
            #plot_keypoints(x, y)
            #break
            
        x_array = np.array(x_array)
        y_array = np.array(y_array)

        print(x_array.shape)
        #output = np.array(output)
        #output = output.transpose(1, 0, 2)  
      
        output = cal_vector_change(x_array, y_array)

        result.append(output)
 
    # print("LEN")
    # for l in result : 
    #  print(len(l))
    # print("END")
    result = np.array(result)

    # print("calculate")
    # print(result.shape)
    return result

print(X_train_names[0])
calculate_change_keypoints([X_train_names[0]], 0, 'right')
#calculate_change_keypoints(X_train_names, 0, 'right')

squat_good_7.npy
(49, 7)


array([[0.25626787, 0.29660416, 0.26835613, 0.20754207, 0.11111432,
        0.52031973, 0.36614922, 0.75049154, 0.3681233 , 0.57100982,
        0.30102785, 0.26799525, 0.15538296, 0.42836036, 0.19944582,
        0.32131047, 0.10271608, 0.36794502, 0.40651869, 0.26924676,
        0.18470168, 0.36263136, 0.09118946, 0.08535037, 0.31594329,
        0.22921109, 0.18062867, 0.2806716 , 0.19331895, 0.34991389,
        0.34737182, 0.20885604, 0.26396624, 0.54433309, 0.30970876,
        0.35603953, 0.15353657, 0.81690241, 0.15965961, 0.2092375 ,
        0.18263194, 0.24328682, 0.23860944, 0.17337322, 0.24011458,
        0.24257443, 0.17237461, 0.26299327]])

In [120]:
def KNN_with_keypoints(X_trains, X_test):
    predictions = []
    train_d_vectors = calculate_change_keypoints(X_train_names, 0)
    test_d_vector   = calculate_change_keypoints([X_test], 1)
    
    good, bad = [[] for i in range(2)]

    for idx, train_d_vector in enumerate(train_d_vectors): # X_test_names
        # Store the average distance to good and bad training examples
        #print("SHAPE")
        #print(len(train_d_vector)
        #print(len(test_d_vector))
        #print(train_d_vector)
        #print(test_d_vector)
        #test_d_vector = test_d_vector.reshape(-1, 1)
        
        dist = utils.DTWDistance(train_d_vector, test_d_vector[0])
        
        if y_train[idx]:
            good.append(dist)
        else:
            bad.append(dist)
              
        # if len(good) == 0 : good = [0]
        # if len(bad) == 0 : bad = [0]
        
    # 평가시 Train에 good, bad를 모두 넣어줘야함 
    good_score = np.mean(good)
    bad_score  = np.mean(bad)
    
    print(good_score, bad_score)
    
    #print(good_score, bad_score)
    # dist가 크면 덜 가깝다 -> bad score가 크면 good에 가깝다
    if good_score < bad_score:
        return 1
    else:
        return 0
    
result = []
for X_test_name in X_test_names :
    print(X_test_name, end=" ")
    test_label = KNN_with_keypoints(X_test_names, X_test_name) # 기존 test data 
    print(test_label)
    
    result.append([X_test_name, test_label])

squat_good_2.npy (49, 7)
(35, 7)
(44, 7)
(39, 7)
(139, 7)
(62, 7)
(34, 7)
(67, 7)
(99, 7)
63.061816019178956 41.91114054474744
0
squat_bad_6.npy (49, 7)
(35, 7)


/Users/imjungsun/.pyenv/versions/urop_env/lib/python3.6/site-packages/ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


(44, 7)
(39, 7)
(139, 7)
(62, 7)
(34, 7)
(67, 7)
(36, 7)
49.14088922915378 0.890624265638139
0
squat_good_6.npy (49, 7)
(35, 7)
(44, 7)
(39, 7)
(139, 7)
(62, 7)
(34, 7)
(67, 7)
(35, 7)
48.49739800597456 1.0650758608737936
0


In [121]:
print("## TRAIN SET ##")
for i in range(len(X_train_names)) : 
    print(X_train_names[i], end = " ")
    if y_train[i] == 1 :
        print("Good")
    else :
        print("Bad")
        
        
print("\n## TEST RESULT ##")
for r in result : 
    ori_result = r[0].split("_")[1]
    
    print(r[0], end = " ")
    if r[1] == 1 :
        print("Good", end = " ")
    else :
        print("Bad", end = " ")

    if ori_result == 'good' and r[1] == 1 : 
        print("Right")  
    
    elif ori_result == 'bad' and r[1] == 0 : 
        print("Right")
        
    else : print("Wrong")

## TRAIN SET ##
squat_good_7.npy Good
squat_bad_8.npy Bad
squat_bad_7.npy Bad
squat_good_5.npy Good
squat_good_1.npy Good
squat_good_4.npy Good
squat_bad_9.npy Bad
squat_good_3.npy Good

## TEST RESULT ##
squat_good_2.npy Bad Wrong
squat_bad_6.npy Bad Right
squat_good_6.npy Bad Wrong
